In [2]:
!pip install streamlit streamlit_chat langchain  faiss-gpu tiktoken pypdf sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 24.9 MB/s et

In [3]:
!pip install peft accelerate bitsandbytes trl safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


In [1]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from torch import cuda, bfloat16
import transformers
import torch
from transformers import BitsAndBytesConfig , AutoConfig , AutoModelForCausalLM , pipeline , AutoTokenizer



In [11]:
def evalution_function(question:str,
                       answer:str,
                       position:str,
                       hf_auth:str,
                       mistral_hugging_face_id: str,
                       temperature_scale: float = 0.4,
                       ):
  """
  args:
    quetion: The generated question from our database
    answer: answer given by the candidate
    position: job position that the candidate applying for
    temperature scale: to conrol randomness in the generated text
    hf_auth : your hugging face authentication's token
    model_id : mistrl 7b instruct id from huggingface
  """
  model_id = mistral_hugging_face_id
  use_flash_attention = False

  # set quantization configuration to load large model with less GPU memory
  # this requires the `bitsandbytes` library
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=bfloat16
          )

  # begin initializing HF items, need auth token for these
  auth = hf_auth
  use_flash_attention = False
  #mistral 7b instruct config
  model_config = AutoConfig.from_pretrained(
      model_id,
      token=hf_auth
          )
  #loading mistral model with 4bit qunatization
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      trust_remote_code=True,
      config=model_config,
      quantization_config=bnb_config,
      use_flash_attention_2=use_flash_attention,

      device_map='auto',
      torch_dtype=torch.float16,
      token= auth
        )
  #mistral 7b tokenizer
  tokenizer = AutoTokenizer.from_pretrained(
      model_id,
      token=auth
          )

  # intializing text generation pipeline
  generate_text = pipeline(
      model=model, tokenizer=tokenizer,
      return_full_text=True,  # langchain expects the full text
      task='text-generation',
      # we pass model parameters here too
      temperature=temperature_scale,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
      max_new_tokens=2024,  # mex number of tokens to generate in the output
      repetition_penalty=1.1, # without this output begins repeating
      pad_token_id= 50256
                  )

  llm = HuggingFacePipeline(pipeline=generate_text)

  #prompt for evaluating the candidate's answer
  prompt = ChatPromptTemplate.from_template(
  """### instruction: you are an experienced interviewer.\
   You are interviewing a candidate for the position of {position} .\
   You are tasked to rate an answer provided by the candidate. You should provide a categorical rating and qualitative feedback.\
    The categorical rating should be one of the following values: Good, average, or  Poor.\
      the qualitative feedback should provide sufficient details to justify the categorical rating.\
        the format instructions of the output and the question asked to the candidate and the answer given by the candidate are  given below.\
        ### format instruction: {format_instructions}.\
        ### question:{question}.\
        ### answer:{answer}.\
        ### Rating:
        """
      )

  #using single chain
  chain = LLMChain(llm=llm, prompt=prompt)

  """
  formating the output to JSON:
    {"Rating": "rating provided by LLM"
      "qualitative_feedback":"feedback provided by LLM}
  """
  #defining Rating schema
  Rating_schema = ResponseSchema(name="Rating",
                             description="it was the categorical value for the answer given by the candidate and this value could be poor, average or good. \
                                             ,the categorical value given by you as experienced interviewer. \
                                            after asking a candidate a question related to the position he is appling for")
  #defining feedback schema
  qualitative_feedback_schema = ResponseSchema(name="qualitative_feedback",
                                      description="the qualitative feedback is the sufficient details  which is given by you as an Experienced interviewr. \
                                          the qualitative feedback is given after asking the candidate a question related the the position he is appling for, \
                                           and the candidate provided his answer. \
                                            the qualitative feedback should privide sufficent details to justify the categorical rating ")

  #stacking the two schemas
  response_schemas = [Rating_schema,
                      qualitative_feedback_schema]

  #parsing the output
  output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

  #extrating format instructions
  format_instructions = output_parser.get_format_instructions()

  #dictionary for mapping the question, answer, position and fromat_instructions to the prompt
  question_answer = {"question": question,
                     "answer": answer,
                     "position": position,
                     "format_instructions":format_instructions }

  #running the chain
  response = chain.run(question_answer)

  #output dictionary having two keys "Rating" and "qualitative_feedback"
  output_dict = output_parser.parse(response)


  return output_dict["Rating"],output_dict["qualitative_feedback"]


In [12]:
Rating , feedback = evalution_function("How can you improve a dissatisfied customer's experience?",
                                        "I've found the most successful strategy for turning an unhappy customer into a happy customer is by actively listening to what they're saying. Sometimes, customers just want you to listen to them, and they want to feel like the company cares about them and their opinions.",
                                        "customer service representative",
                                        '',
                                        'mistralai/Mistral-7B-Instruct-v0.1',
                                        0.5,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [13]:
Rating

'Good'

In [14]:
feedback

'The candidate provided a clear and concise answer that highlights the importance of active listening in resolving customer issues. The answer also shows empathy towards the customer and acknowledges the need for the company to care about their opinions.'